In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_curve, auc

In [9]:
# Load the data
tic_data = pd.read_table('2023-08-18-IS-tic.csv', sep=',', header=None)
mix_data = pd.read_csv('2023-08-18-MixDat.csv')

In [34]:
# Select one-tenth of the rows from both classes for the subset
subset_tic_data = pd.concat([tic_data[tic_data.index % 10 == i] for i in range(10)])
subset_mix_data = pd.concat([mix_data[mix_data.index % 10 == i] for i in range(10)])

In [35]:
# Split the data into training and test sets
tic_train, tic_test, mix_train, mix_test = train_test_split(subset_tic_data, subset_mix_data, test_size=0.2, random_state=42)


In [37]:
# Extract labels from mix_train and mix_test
y_train = mix_train['class']
y_test = mix_test['class']

In [38]:
# Scale and center the columns using the mean and standard deviation from the training set
scaler = StandardScaler()

In [39]:
# Fit the scaler on the training data and transform both training and test data
tic_train_scaled = scaler.fit_transform(tic_train)
tic_test_scaled = scaler.transform(tic_test)

In [40]:
# Fit the scaler on the training data and transform both training and test data
tic_train_scaled = scaler.fit_transform(tic_train)
tic_test_scaled = scaler.transform(tic_test)

In [41]:
# Convert the scaled data back to pandas DataFrame
tic_train_scaled_df = pd.DataFrame(tic_train_scaled, columns=tic_train.columns)
tic_test_scaled_df = pd.DataFrame(tic_test_scaled, columns=tic_test.columns)

Logistic Regression

In [44]:
Logit = LogisticRegressionCV(cv=10, scoring='accuracy', n_jobs=-1, max_iter = 100)

In [45]:
Logit.fit(tic_train_scaled_df, np.ravel(y_train))
y_pred = Logit.predict(tic_test_scaled_df)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

In [48]:
pd.DataFrame(data = Logit.coef_, columns = tic_data.columns)

,0,1,2,3,4,5,6,7,8,9,...,2790,2791,2792,2793,2794,2795,2796,2797,2798,2799
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names="class"))

[[4967   39]
 [   0 4994]]


InvalidParameterError: The 'target_names' parameter of classification_report must be an array-like or None. Got 'class' instead.

In [59]:
from sklearn import metrics

In [63]:
fpr, tpr, _ = metrics.roc_curve(y_test, Logit.predict_proba(tic_test_scaled_df))

ValueError: y should be a 1d array, got an array of shape (10000, 2) instead.

In [54]:
from sklearn.metrics import accuracy_score

In [55]:
accuracy_score(y_test, y_pred)

0.9961

In [58]:
auc_logit = pd.DataFrame(data = {"fpr":fpr,"tpr":tpr})

NameError: name 'fpr' is not defined

In [ ]:
plt.figure(figsize = (15,10))
plot = sns.lineplot(x='fpr', y='tpr', data=auc_logit).set(title='AUC ROC LOGIT')

LDA

In [65]:
LDA = LinearDiscriminantAnalysis()

In [66]:
LDA.fit(tic_train_scaled_df, np.ravel(y_train))
y_pred = LDA.predict(tic_test_scaled_df)

In [69]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names='class'))

[[4169  837]
 [   5 4989]]


InvalidParameterError: The 'target_names' parameter of classification_report must be an array-like or None. Got 'class' instead.

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  LDA.predict_proba(tic_test_scaled_df)[:,1])

In [ ]:
metrics.auc(fpr, tpr)

In [72]:
accuracy_score(y_test, y_pred)

0.9158

In [ ]:
auc_LDA = pd.DataFrame(data = {"fpr":fpr,"tpr":tpr})

In [ ]:
plt.figure(figsize = (15,10))
plot = sns.lineplot(x='fpr', y='tpr', data=auc_LDA).set(title='AUC ROC LDA')

KNN

In [73]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}

gs = GridSearchCV(
    KNeighborsClassifier(),
    param_grid={"n_neighbors": range(1, 21), "weights":["uniform","distance"], "p":[1, 2]},
    scoring=scoring,
    refit="AUC",
    return_train_score=True,
    n_jobs = -1,
    cv = 10,
    verbose = 3
)
gs.fit(tic_train_scaled_df, np.ravel(y_train))
results = gs.cv_results_

Fitting 10 folds for each of 80 candidates, totalling 800 fits


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 459, in _score
    y_pred = method_caller(clf, "decision_function", X, pos_label=pos_label)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 86, in _cached_call
    result, _ = _get_response_values(
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/_response.py", line 73, in _get_response_values
    prediction_method = _check_response_method(estimator, response_method)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/v

KeyboardInterrupt: 

In [ ]:
gs.best_params_

In [74]:
KNN = KNeighborsClassifier(n_neighbors=20, p = 2, weights = "distance")
KNN.fit(tic_train_scaled_df, np.ravel(y_train))
y_pred = KNN.predict(tic_test_scaled_df)

In [75]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=Class))

[[3112 1894]
 [   9 4985]]


NameError: name 'Class' is not defined

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  KNN.predict_proba(tic_test_scaled_df)[:,1])

In [ ]:
metrics.auc(fpr, tpr)

In [76]:
accuracy_score(y_test, y_pred)

0.8097

In [ ]:
auc_KNN = pd.DataFrame(data = {"fpr":fpr,"tpr":tpr})

In [ ]:
plt.figure(figsize = (15,10))
plot = sns.lineplot(x='fpr', y='tpr', data=auc_KNN).set(title='AUC ROC KNN')

Linear SVM

In [77]:
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

In [78]:
from sklearn.model_selection import StratifiedShuffleSplit

In [79]:
cv = StratifiedShuffleSplit(n_splits=30, test_size=0.2, random_state=101)

In [80]:
grid = GridSearchCV(SVC(kernel = "linear", max_iter=10000, probability=True), param_grid=param_grid, cv=cv, n_jobs = -1, verbose = 3)

In [81]:
grid.fit(tic_train_scaled_df, np.ravel(y_train))

Fitting 30 folds for each of 13 candidates, totalling 390 fits


KeyboardInterrupt: 

In [ ]:
results = grid.cv_results_

In [ ]:
LSVC = SVC(kernel = "linear", max_iter=100, C = 0.1, probability = True)
LSVC.fit(tic_train_scaled_df, np.ravel(y_train))
y_pred = LSVC.predict(tic_test_scaled_df)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=Class))

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  LSVC.predict_proba(X_test)[:,1])

In [ ]:
metrics.auc(fpr, tpr)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
auc_LSVC = pd.DataFrame(data = {"fpr":fpr,"tpr":tpr})

In [ ]:
plt.figure(figsize = (15,10))
plot = sns.lineplot(x='fpr', y='tpr', data=auc_LSVC).set(title='AUC ROC LINEAR SVC')

Radial SVC

In [ ]:
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=101)
grid = GridSearchCV(SVC(probability = True), param_grid=param_grid, cv=cv, n_jobs = -1, verbose = 3)

In [ ]:
grid.fit(X_train, np.ravel(y_train))

In [ ]:
results = grid.cv_results_

In [ ]:
grid.best_params_

In [ ]:
RSVC = SVC(C = 100, gamma = 0.1, probability = True)
RSVC.fit(X_train, np.ravel(y_train))
y_pred = RSVC.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=Class))

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test,  RSVC.predict_proba(X_test)[:,1])

In [ ]:
metrics.auc(fpr, tpr)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
auc_RSVC = pd.DataFrame(data = {"fpr":fpr,"tpr":tpr})

In [ ]:
plt.figure(figsize = (15,10))
plot = sns.lineplot(x='fpr', y='tpr', data=auc_RSVC).set(title='AUC ROC RADIAL SVC')